*** Changed my topic because I could not find JSON file for  Vienna ***

<h1 align="center">Clustering Neighbourhoods of London and Paris </h1>
 <h2 align="center">Capstone Project - IBM Data Science Professional Certificate </h2>

<h2 align="left">Introduction </h2> 

London and Paris are two of the most popular touristic city in Europe. 

For the first time in 10 years, London has been beaten by Paris in the battle for visitors. The French capital received the highest number of tourists of any European city in 2019, attracting 19.1 million foreign visitors. Despite Paris’s tourism growth, visitor spending was still higher in London: overnight international visitor spend for the latter was 16.5 billion dollar last year, compared to 14 billion dollar in the former.

The objective of this project is to group the attarctions of London and Paris respectively and draw insights to explain the growth in tourisim in Paris and the higher reveneu generated by London. 


<h2 align="left">Data </h2>

We require geolocation data for both London and Paris. Using Postal codes we use can find out the neighbourhoods, boroughs, venues and their most popular venue categories.

<h4 align="left">Paris </h4>

We scrape JSON data available at https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e

The JSON file has data about all the neighbourhoods in France but we are only interested in Paris

postal_code : Postal codes for France
nom_comm : Name of Neighbourhoods in France
nom_dept : Name of the boroughs, equivalent to towns in France
geo_point_2d : Tuple containing the latitude and longitude of the Neighbourhoods.

<h4 align="left">London </h4>

We scrape our data from https://en.wikipedia.org/wiki/List_of_areas_of_London

This wikipedia page has information about all the neighbourhoods,but we are only interested in London.

borough : Name of Neighbourhood
town : Name of borough
post_code : Postal codes for London.
This wikipedia page lacks information about the geographical locations. To solve this problem we use ArcGIS API


<h4 align="left">Foursquare API Data </h4>

Foursquare is a location data provider with information about all manner of venues and events within an area of interest. Such information includes venue names, locations, menus and even photos. As such, the foursquare location platform will be used as the sole data source since all the stated required information can be obtained through the API.

After finding the list of neighbourhoods, we then connect to the Foursquare API to gather information about venues inside each and every neighbourhood. For each neighbourhood, we have chosen the radius to be 500 meters.


<h4 align="left">ArcGIS API </h4>

ArcGIS Online enables you to connect people, locations, and data using interactive maps. Work with smart, data-driven styles and intuitive analysis tools that deliver location intelligence. Share your insights with the world or specific groups.

More specifically, we use ArcGIS to get the geo locations of the neighbourhoods of London. The following columns are added to our initial dataset which prepares our data.







<h2 align="left">Import Libraries & API </h2>

In [135]:
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import json
# import k-means for the clustering stage
from sklearn.cluster import KMeans

<h3 align="left">ArcGIS API </h3>

In [5]:
# pip install arcgis
from arcgis.geocoding import geocode
from arcgis.gis import GIS
gis = GIS()

<h3 align="left">Foursquare API  </h3>

In [57]:
CLIENT_ID = 'RUVAK431FJHKYXW53PBRRL5WRWQ4YPXYNAE4PTFHS3OCHJCC' # your Foursquare ID
CLIENT_SECRET = 'IDK2UJ5HUXHJYSLGUNQC13Z0PZJVE21TN01MB2RN2ACHKVSF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RUVAK431FJHKYXW53PBRRL5WRWQ4YPXYNAE4PTFHS3OCHJCC
CLIENT_SECRET:IDK2UJ5HUXHJYSLGUNQC13Z0PZJVE21TN01MB2RN2ACHKVSF


<h2 align="left">Exploring Paris </h2>

In [7]:
# Read the JSON file
paris_raw=pd.read_json("correspondances-code-insee-code-postal.json")
paris_raw.head()

,datasetid,recordid,fields,geometry,record_timestamp
0,correspondances-code-insee-code-postal,2bf36b38314b6c39dfbcd09225f97fa532b1fc45,"{'code_comm': '645', 'nom_dept': 'ESSONNE', 's...","{'type': 'Point', 'coordinates': [2.2517129721...",2016-09-21T00:29:06.175+02:00
1,correspondances-code-insee-code-postal,7ee82e74e059b443df18bb79fc5a19b1f05e5a88,"{'code_comm': '133', 'nom_dept': 'SEINE-ET-MAR...","{'type': 'Point', 'coordinates': [3.0529405055...",2016-09-21T00:29:06.175+02:00
2,correspondances-code-insee-code-postal,e2cd3186f07286705ed482a10b6aebd9de633c81,"{'code_comm': '378', 'nom_dept': 'ESSONNE', 's...","{'type': 'Point', 'coordinates': [2.1971816504...",2016-09-21T00:29:06.175+02:00
3,correspondances-code-insee-code-postal,868bf03527a1d0a9defe5cf4e6fa0a730d725699,"{'code_comm': '243', 'nom_dept': 'SEINE-ET-MAR...","{'type': 'Point', 'coordinates': [2.7097808131...",2016-09-21T00:29:06.175+02:00
4,correspondances-code-insee-code-postal,21e809b1d4480333c8b6fe7addd8f3b06f343e2c,"{'code_comm': '003', 'nom_dept': 'VAL-DE-MARNE...","{'type': 'Point', 'coordinates': [2.3335102498...",2016-09-21T00:29:06.175+02:00


In [8]:
# Break down each of the nested fields and create the dataframe that we need
paris_field_data = pd.DataFrame()
for f in paris_raw.fields:
    dict_new = f
    paris_field_data = paris_field_data.append(dict_new, ignore_index=True)

paris_field_data.head()

,code_arr,code_cant,code_comm,code_dept,code_reg,geo_point_2d,geo_shape,id_geofla,insee_com,nom_comm,nom_dept,nom_region,population,postal_code,statut,superficie,z_moyen
0,3,03,645,91,11,"[48.750443119964764, 2.251712972144151]","{'type': 'Polygon', 'coordinates': [[[2.238024...",16275,91645,VERRIERES-LE-BUISSON,ESSONNE,ILE-DE-FRANCE,15.5,91370,Commune simple,999.0,121.0
1,3,20,133,77,11,"[48.41256065214989, 3.052940505560729]","{'type': 'Polygon', 'coordinates': [[[3.076046...",31428,77133,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,ILE-DE-FRANCE,0.2,77126,Commune simple,1082.0,88.0
2,1,09,378,91,11,"[48.52726809075556, 2.19718165044305]","{'type': 'Polygon', 'coordinates': [[[2.203466...",30975,91378,MAUCHAMPS,ESSONNE,ILE-DE-FRANCE,0.3,91730,Commune simple,313.0,150.0
3,5,14,243,77,11,"[48.87307018579678, 2.7097808131278462]","{'type': 'Polygon', 'coordinates': [[[2.727542...",17000,77243,LAGNY-SUR-MARNE,SEINE-ET-MARNE,ILE-DE-FRANCE,20.2,77400,Chef-lieu canton,579.0,71.0
4,3,34,003,94,11,"[48.80588035965699, 2.333510249842654]","{'type': 'Polygon', 'coordinates': [[[2.343851...",32123,94003,ARCUEIL,VAL-DE-MARNE,ILE-DE-FRANCE,19.5,94110,Chef-lieu canton,232.0,70.0


In [9]:
# Take the columns geo_point_2d, nom_dept, nom_comm and postal_code
df_2 = paris_field_data[['postal_code','nom_comm','nom_dept','geo_point_2d']]
df_2

,postal_code,nom_comm,nom_dept,geo_point_2d
0,91370,VERRIERES-LE-BUISSON,ESSONNE,"[48.750443119964764, 2.251712972144151]"
1,77126,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,"[48.41256065214989, 3.052940505560729]"
2,91730,MAUCHAMPS,ESSONNE,"[48.52726809075556, 2.19718165044305]"
3,77400,LAGNY-SUR-MARNE,SEINE-ET-MARNE,"[48.87307018579678, 2.7097808131278462]"
4,94110,ARCUEIL,VAL-DE-MARNE,"[48.80588035965699, 2.333510249842654]"
...,...,...,...,...
1295,77520,CESSOY-EN-MONTOIS,SEINE-ET-MARNE,"[48.50730730461658, 3.138844194183689]"
1296,93420,VILLEPINTE,SEINE-SAINT-DENIS,"[48.95902025378707, 2.536306342059409]"
1297,77130,CANNES-ECLUSE,SEINE-ET-MARNE,"[48.36403767307805, 2.990786679832767]"
1298,78930,VILLETTE,YVELINES,"[48.92627887061508, 1.6937417245662671]"


In [10]:
# We localize to only include Paris
df_paris = df_2[df_2['nom_dept'].str.contains('PARIS')].reset_index(drop=True)
df_paris

,postal_code,nom_comm,nom_dept,geo_point_2d
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,"[48.87689616237872, 2.337460241388529]"
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,"[48.86790337886785, 2.344107166658533]"
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,"[48.85941549762748, 2.378741060237548]"
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,"[48.86305413181178, 2.359361058970589]"
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,"[48.84896809191946, 2.332670898588416]"
5,75019,PARIS-19E-ARRONDISSEMENT,PARIS,"[48.88686862295828, 2.384694327870042]"
6,75020,PARIS-20E-ARRONDISSEMENT,PARIS,"[48.86318677744551, 2.400819826729021]"
7,75015,PARIS-15E-ARRONDISSEMENT,PARIS,"[48.84015541860987, 2.293559372435076]"
8,75005,PARIS-5E-ARRONDISSEMENT,PARIS,"[48.844508659617546, 2.349859385560182]"
9,75007,PARIS-7E-ARRONDISSEMENT,PARIS,"[48.85608259819694, 2.312438687733857]"


In [11]:
df_paris.shape

(20, 4)

In [12]:
df_paris.dtypes

postal_code     object
nom_comm        object
nom_dept        object
geo_point_2d    object
dtype: object

In [13]:
df_paris['geo_point_2d'][0] # Checking one of the geo coordinates.

[48.87689616237872, 2.337460241388529]

In [14]:
temp1 = df_paris['geo_point_2d']
temp1

0      [48.87689616237872, 2.337460241388529]
1      [48.86790337886785, 2.344107166658533]
2      [48.85941549762748, 2.378741060237548]
3      [48.86305413181178, 2.359361058970589]
4      [48.84896809191946, 2.332670898588416]
5      [48.88686862295828, 2.384694327870042]
6      [48.86318677744551, 2.400819826729021]
7      [48.84015541860987, 2.293559372435076]
8     [48.844508659617546, 2.349859385560182]
9      [48.85608259819694, 2.312438687733857]
10     [48.87252726662346, 2.312582560420059]
11     [48.82871768452136, 2.362468228516128]
12     [48.83515623066034, 2.419807034965275]
13    [48.892735074561706, 2.348711933867703]
14      [48.8626304851685, 2.336293446550539]
15     [48.87602855694339, 2.361112904561707]
16     [48.86039876035177, 2.262099559395783]
17    [48.854228281954754, 2.357361938142205]
18     [48.88733716648682, 2.307485559493426]
19     [48.82899321160942, 2.327100883257538]
Name: geo_point_2d, dtype: object

In [15]:
paris_latlng = df_paris['geo_point_2d'].astype('str')

In [16]:
# Spliting the geo_point_2d column into latitude and longitude
paris_lat = paris_latlng.apply(lambda x: x.split(',')[0])
paris_lat = paris_lat.apply(lambda x: x.lstrip('['))
paris_lat

0      48.87689616237872
1      48.86790337886785
2      48.85941549762748
3      48.86305413181178
4      48.84896809191946
5      48.88686862295828
6      48.86318677744551
7      48.84015541860987
8     48.844508659617546
9      48.85608259819694
10     48.87252726662346
11     48.82871768452136
12     48.83515623066034
13    48.892735074561706
14      48.8626304851685
15     48.87602855694339
16     48.86039876035177
17    48.854228281954754
18     48.88733716648682
19     48.82899321160942
Name: geo_point_2d, dtype: object

In [17]:
# Spliting the geo_point_2d column into latitude and longitude
paris_lng = paris_latlng.apply(lambda x: x.split(',')[1])
paris_lng = paris_lng.apply(lambda x: x.rstrip(']'))
paris_lng

0      2.337460241388529
1      2.344107166658533
2      2.378741060237548
3      2.359361058970589
4      2.332670898588416
5      2.384694327870042
6      2.400819826729021
7      2.293559372435076
8      2.349859385560182
9      2.312438687733857
10     2.312582560420059
11     2.362468228516128
12     2.419807034965275
13     2.348711933867703
14     2.336293446550539
15     2.361112904561707
16     2.262099559395783
17     2.357361938142205
18     2.307485559493426
19     2.327100883257538
Name: geo_point_2d, dtype: object

In [18]:
paris_geo_lat  = pd.DataFrame(paris_lat.astype(float))
paris_geo_lat.columns=['Latitude']
paris_geo_lng = pd.DataFrame(paris_lng.astype(float))
paris_geo_lng.columns=['Longitude']

In [19]:
#  Drop the geo_point_2d column from our previously stored df_paris and concatenating with the latitude and longitude 
paris_combined_data = pd.concat([df_paris.drop('geo_point_2d', axis=1), paris_geo_lat, paris_geo_lng], axis=1)
paris_combined_data

,postal_code,nom_comm,nom_dept,Latitude,Longitude
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.863054,2.359361
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.848968,2.332671
5,75019,PARIS-19E-ARRONDISSEMENT,PARIS,48.886869,2.384694
6,75020,PARIS-20E-ARRONDISSEMENT,PARIS,48.863187,2.400820
7,75015,PARIS-15E-ARRONDISSEMENT,PARIS,48.840155,2.293559
8,75005,PARIS-5E-ARRONDISSEMENT,PARIS,48.844509,2.349859
9,75007,PARIS-7E-ARRONDISSEMENT,PARIS,48.856083,2.312439


In [20]:
# Co-ordinates for Paris
paris = geocode(address='Paris, France, FR')[0]
paris_lng_coords = paris['location']['x']
paris_lat_coords = paris['location']['y']
print("The geolocation of Paris: ", paris_lat_coords, paris_lng_coords)

The geolocation of Paris:  48.85341000000005 2.3488000000000397


In [21]:
# Creating the map of Paris
map_Paris= folium.Map(location=[paris_lat_coords, paris_lng_coords], zoom_start=12)
map_Paris

# adding markers to map
for latitude, longitude, borough, town in zip(paris_combined_data['Latitude'], paris_combined_data['Longitude'], paris_combined_data['nom_comm'], paris_combined_data['nom_dept']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='Blue',
        fill=True,
        fill_opacity=0.8
        ).add_to(map_Paris)  
    
map_Paris

In [22]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
# Let's get the neaby venues present in each neighbourhood of Paris
venues_in_Paris = getNearbyVenues(paris_combined_data['nom_comm'], paris_combined_data['Latitude'], paris_combined_data['Longitude'])

PARIS-9E-ARRONDISSEMENT
PARIS-2E-ARRONDISSEMENT
PARIS-11E-ARRONDISSEMENT
PARIS-3E-ARRONDISSEMENT
PARIS-6E-ARRONDISSEMENT
PARIS-19E-ARRONDISSEMENT
PARIS-20E-ARRONDISSEMENT
PARIS-15E-ARRONDISSEMENT
PARIS-5E-ARRONDISSEMENT
PARIS-7E-ARRONDISSEMENT
PARIS-8E-ARRONDISSEMENT
PARIS-13E-ARRONDISSEMENT
PARIS-12E-ARRONDISSEMENT
PARIS-18E-ARRONDISSEMENT
PARIS-1ER-ARRONDISSEMENT
PARIS-10E-ARRONDISSEMENT
PARIS-16E-ARRONDISSEMENT
PARIS-4E-ARRONDISSEMENT
PARIS-17E-ARRONDISSEMENT
PARIS-14E-ARRONDISSEMENT


In [24]:
venues_in_Paris.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Le Bouclier de Bacchus,48.876834,2.337843,Wine Bar
1,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Place Saint-Georges,48.878408,2.337575,Plaza
2,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,RAP,48.876628,2.339359,Gourmet Shop
3,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Aji Dulce,48.877906,2.337878,Venezuelan Restaurant
4,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,So Nat,48.876277,2.338614,Vegetarian / Vegan Restaurant


In [25]:
venues_in_Paris.shape

(1307, 7)

In [26]:
# How many Venue Categories are there for each neighbourhood
venues_in_Paris.groupby('Venue Category').max()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Afghan Restaurant,PARIS-11E-ARRONDISSEMENT,48.859415,2.378741,Afghanistan,48.862327,2.379999
African Restaurant,PARIS-9E-ARRONDISSEMENT,48.876896,2.361113,Wally Le Saharien,48.879211,2.357852
American Restaurant,PARIS-19E-ARRONDISSEMENT,48.892735,2.384694,Harper's,48.894373,2.379129
Antique Shop,PARIS-9E-ARRONDISSEMENT,48.876896,2.337460,Hôtel des Ventes Drouot,48.873061,2.340101
Argentinian Restaurant,PARIS-3E-ARRONDISSEMENT,48.863054,2.359361,Anahi,48.867002,2.358244
...,...,...,...,...,...,...
Wine Bar,PARIS-9E-ARRONDISSEMENT,48.892735,2.400820,Vingt Vins d'Art,48.891257,2.397696
Wine Shop,PARIS-3E-ARRONDISSEMENT,48.876029,2.361113,Trois Fois Vin,48.879651,2.363401
Women's Store,PARIS-6E-ARRONDISSEMENT,48.867903,2.344107,L'Appartement Sézane,48.869574,2.345060


In [27]:
# One Hot Encoding: We need to encode our venue categories to get a better result for our clustering
Paris_venue_cat = pd.get_dummies(venues_in_Paris[['Venue Category']], prefix="", prefix_sep="")
Paris_venue_cat

,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1303,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1304,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1305,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Adding Neighbourhoods to our Data
Paris_venue_cat['Neighbourhood'] = venues_in_Paris['Neighbourhood'] 

# Moving neighborhood column to the first column
fixed_columns = [Paris_venue_cat.columns[-1]] + list(Paris_venue_cat.columns[:-1])
Paris_venue_cat = Paris_venue_cat[fixed_columns]

Paris_venue_cat.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [29]:
# Group the Neighbourhoods and calculate the mean venue categories value in each Neighbourhood
Paris_grouped = Paris_venue_cat.groupby('Neighbourhood').mean().reset_index()
Paris_grouped.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,PARIS-10E-ARRONDISSEMENT,0.00000,0.02,0.0,0.0,0.0,0.0,0.00000,0.0,0.030000,...,0.0,0.0,0.010000,0.0,0.000000,0.02000,0.02,0.0,0.0,0.0
1,PARIS-11E-ARRONDISSEMENT,0.02381,0.00,0.0,0.0,0.0,0.0,0.02381,0.0,0.023810,...,0.0,0.0,0.047619,0.0,0.023810,0.02381,0.00,0.0,0.0,0.0
2,PARIS-12E-ARRONDISSEMENT,0.00000,0.00,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.00,0.0,0.2,0.2
3,PARIS-13E-ARRONDISSEMENT,0.00000,0.00,0.0,0.0,0.0,0.0,0.00000,0.0,0.189655,...,0.0,0.0,0.000000,0.0,0.206897,0.00000,0.00,0.0,0.0,0.0
4,PARIS-14E-ARRONDISSEMENT,0.00000,0.00,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.00,0.0,0.0,0.0


In [79]:
# create a new dataframe for Paris
neighborhoods_venues_sorted_paris = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_paris['Neighbourhood'] = Paris_grouped['Neighbourhood']

for ind in np.arange(Paris_grouped.shape[0]):
    neighborhoods_venues_sorted_paris.iloc[ind, 1:] = return_most_common_venues(Paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_paris.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-10E-ARRONDISSEMENT,French Restaurant,Hotel,Coffee Shop,Café,Bistro,Japanese Restaurant,Pizza Place,Indian Restaurant,Italian Restaurant,Asian Restaurant
1,PARIS-11E-ARRONDISSEMENT,Restaurant,Café,French Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar,Bakery,Bistro,Pastry Shop,Afghan Restaurant
2,PARIS-12E-ARRONDISSEMENT,Zoo Exhibit,Supermarket,Bistro,Monument / Landmark,Zoo,Indie Movie Theater,Escape Room,Fountain,Food & Drink Shop,Flower Shop
3,PARIS-13E-ARRONDISSEMENT,Vietnamese Restaurant,Asian Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Juice Bar,Hotel,Dessert Shop,Coffee Shop,Gourmet Shop
4,PARIS-14E-ARRONDISSEMENT,French Restaurant,Hotel,Japanese Restaurant,Food & Drink Shop,Café,Bistro,Bakery,Fast Food Restaurant,Pizza Place,Plaza


In [80]:
# set number of clusters
k_num_clusters = 5

Paris_grouped_clustering = Paris_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_Paris = KMeans(n_clusters=k_num_clusters, random_state=0).fit(Paris_grouped_clustering)
kmeans_Paris

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [81]:
# Labelling Clustered Data
kmeans_Paris.labels_

array([2, 2, 3, 4, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2])

In [82]:
# Insert it in our data.
neighborhoods_venues_sorted_paris.insert(0, 'Cluster Labels', kmeans_Paris.labels_ +1)

In [83]:
paris_data = paris_combined_data

paris_data = paris_data.join(neighborhoods_venues_sorted_paris.set_index('Neighbourhood'), on='nom_comm')

paris_data.head()

,postal_code,nom_comm,nom_dept,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460,3,French Restaurant,Hotel,Japanese Restaurant,Bistro,Lounge,Bakery,Wine Bar,Cocktail Bar,Vegetarian / Vegan Restaurant,Cheese Shop
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107,3,French Restaurant,Cocktail Bar,Italian Restaurant,Wine Bar,Bakery,Bistro,Bar,Thai Restaurant,Salad Place,Hotel
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741,3,Restaurant,Café,French Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar,Bakery,Bistro,Pastry Shop,Afghan Restaurant
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.863054,2.359361,3,Coffee Shop,Bakery,French Restaurant,Bistro,Burger Joint,Sandwich Place,Café,Cocktail Bar,Art Gallery,Japanese Restaurant
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.848968,2.332671,3,French Restaurant,Chocolate Shop,Bistro,Bakery,Pub,Fountain,Italian Restaurant,Athletics & Sports,Plaza,Restaurant


In [84]:
# Drop all the NaN values to prevent data skew
paris_data_nonan = paris_data.dropna(subset=['Cluster Labels'])

In [91]:
map_clusters_paris = folium.Map(location=[paris_lat_coords, paris_lng_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_data_nonan['Latitude'], paris_data_nonan['Longitude'], paris_data_nonan['nom_comm'], paris_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + ' ' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.8
        ).add_to(map_clusters_paris)
        
map_clusters_paris

<h3 align="left">Cluster 1 </h3>

In [86]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 1, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,PARIS-16E-ARRONDISSEMENT,1,Plaza,Lake,French Restaurant,Art Museum,Bus Stop,Bus Station,Park,Boat or Ferry,Pool,Ethiopian Restaurant


<h3 align="left">Cluster 2 </h3>

In [87]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 2, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,PARIS-7E-ARRONDISSEMENT,2,French Restaurant,Hotel,Café,Italian Restaurant,Plaza,Cocktail Bar,Coffee Shop,Art Museum,History Museum,Bistro
10,PARIS-8E-ARRONDISSEMENT,2,French Restaurant,Hotel,Spa,Art Gallery,Theater,Cocktail Bar,Coffee Shop,Resort,Bar,Park
18,PARIS-17E-ARRONDISSEMENT,2,French Restaurant,Hotel,Italian Restaurant,Café,Bakery,Japanese Restaurant,Plaza,Bistro,Restaurant,Burger Joint
19,PARIS-14E-ARRONDISSEMENT,2,French Restaurant,Hotel,Japanese Restaurant,Food & Drink Shop,Café,Bistro,Bakery,Fast Food Restaurant,Pizza Place,Plaza


<h3 align="left">Cluster 3 </h3>

In [88]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 3, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-9E-ARRONDISSEMENT,3,French Restaurant,Hotel,Japanese Restaurant,Bistro,Lounge,Bakery,Wine Bar,Cocktail Bar,Vegetarian / Vegan Restaurant,Cheese Shop
1,PARIS-2E-ARRONDISSEMENT,3,French Restaurant,Cocktail Bar,Italian Restaurant,Wine Bar,Bakery,Bistro,Bar,Thai Restaurant,Salad Place,Hotel
2,PARIS-11E-ARRONDISSEMENT,3,Restaurant,Café,French Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar,Bakery,Bistro,Pastry Shop,Afghan Restaurant
3,PARIS-3E-ARRONDISSEMENT,3,Coffee Shop,Bakery,French Restaurant,Bistro,Burger Joint,Sandwich Place,Café,Cocktail Bar,Art Gallery,Japanese Restaurant
4,PARIS-6E-ARRONDISSEMENT,3,French Restaurant,Chocolate Shop,Bistro,Bakery,Pub,Fountain,Italian Restaurant,Athletics & Sports,Plaza,Restaurant
5,PARIS-19E-ARRONDISSEMENT,3,French Restaurant,Bar,Hotel,Bistro,Beer Bar,Brewery,Supermarket,Seafood Restaurant,Restaurant,Steakhouse
6,PARIS-20E-ARRONDISSEMENT,3,Bistro,Bakery,Plaza,Japanese Restaurant,French Restaurant,Bar,Italian Restaurant,Food & Drink Shop,Hotel,Café
7,PARIS-15E-ARRONDISSEMENT,3,Italian Restaurant,Hotel,French Restaurant,Japanese Restaurant,Lebanese Restaurant,Restaurant,Coffee Shop,Thai Restaurant,Bistro,Park
8,PARIS-5E-ARRONDISSEMENT,3,French Restaurant,Hotel,Italian Restaurant,Bakery,Plaza,Café,Bar,Coffee Shop,Pub,Science Museum
13,PARIS-18E-ARRONDISSEMENT,3,French Restaurant,Bar,Bistro,Café,Restaurant,Italian Restaurant,Pizza Place,Plaza,Bakery,Hotel


<h3 align="left">Cluster 4 </h3>

In [89]:

paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 4, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,PARIS-12E-ARRONDISSEMENT,4,Zoo Exhibit,Supermarket,Bistro,Monument / Landmark,Zoo,Indie Movie Theater,Escape Room,Fountain,Food & Drink Shop,Flower Shop


<h3 align="left">Cluster 5 </h3>

In [90]:

paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 5, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,PARIS-13E-ARRONDISSEMENT,5,Vietnamese Restaurant,Asian Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Juice Bar,Hotel,Dessert Shop,Coffee Shop,Gourmet Shop


<h2 align="left">Exploring London </h2>

In [31]:
# Read the list of areas of london wiki page
url_london = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
wiki_london_url = requests.get(url_london)
wiki_london_url

<Response [200]>

In [32]:
wiki_london_data = pd.read_html(wiki_london_url.text)
wiki_london_data

[                                                   0
 0  Map all coordinates in "Category:Areas of Lond...
 1                 Download coordinates as: KML · GPX,
             Location                     London borough       Post town  \
 0         Abbey Wood              Bexley, Greenwich [7]          LONDON   
 1              Acton  Ealing, Hammersmith and Fulham[8]          LONDON   
 2          Addington                         Croydon[8]         CROYDON   
 3         Addiscombe                         Croydon[8]         CROYDON   
 4        Albany Park                             Bexley  BEXLEY, SIDCUP   
 ..               ...                                ...             ...   
 528         Woolwich                          Greenwich          LONDON   
 529   Worcester Park       Sutton, Kingston upon Thames  WORCESTER PARK   
 530  Wormwood Scrubs             Hammersmith and Fulham          LONDON   
 531          Yeading                         Hillingdon           HAYES   
 

In [33]:
wiki_london_data = wiki_london_data[1]
wiki_london_data

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
...,...,...,...,...,...,...
528,Woolwich,Greenwich,LONDON,SE18,020,TQ435795
529,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,020,TQ225655
530,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,020,TQ225815
531,Yeading,Hillingdon,HAYES,UB4,020,TQ115825


In [34]:
# Remove the spaces in the column titles and then we add _ between words.
wiki_london_data.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
wiki_london_data

,Location,London borough,Post_town,Postcode district,Dial code,OS_grid_ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
...,...,...,...,...,...,...
528,Woolwich,Greenwich,LONDON,SE18,020,TQ435795
529,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,020,TQ225655
530,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,020,TQ225815
531,Yeading,Hillingdon,HAYES,UB4,020,TQ115825


In [35]:
# Drop the locations, dial codes and OS grid
df1 = wiki_london_data.drop( [ wiki_london_data.columns[0], wiki_london_data.columns[4], wiki_london_data.columns[5] ], axis=1)
df1.head()

,London borough,Post_town,Postcode district
0,"Bexley, Greenwich [7]",LONDON,SE2
1,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
2,Croydon[8],CROYDON,CR0
3,Croydon[8],CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"


In [36]:
# Rename the columns
df1.columns = ['borough','town','post_code']
df1

,borough,town,post_code
0,"Bexley, Greenwich [7]",LONDON,SE2
1,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
2,Croydon[8],CROYDON,CR0
3,Croydon[8],CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"
...,...,...,...
528,Greenwich,LONDON,SE18
529,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4
530,Hammersmith and Fulham,LONDON,W12
531,Hillingdon,HAYES,UB4


In [37]:
# Remove the Square brackets [ ] and numbers from the borough column
df1['borough'] = df1['borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
df1

,borough,town,post_code
0,"Bexley, Greenwich",LONDON,SE2
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
2,Croydon,CROYDON,CR0
3,Croydon,CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"
...,...,...,...
528,Greenwich,LONDON,SE18
529,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4
530,Hammersmith and Fulham,LONDON,W12
531,Hillingdon,HAYES,UB4


In [38]:
df1.shape

(533, 3)

In [39]:
#  Focusing on the neighbourhoods of London
df1 = df1[df1['town'].str.contains('LONDON')]
df1

,borough,town,post_code
0,"Bexley, Greenwich",LONDON,SE2
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
6,City,LONDON,EC3
7,Westminster,LONDON,WC2
9,Bromley,LONDON,SE20
...,...,...,...
523,Redbridge,LONDON,"IG8, E18"
524,"Redbridge, Waltham Forest","LONDON, WOODFORD GREEN",IG8
527,Barnet,LONDON,N12
528,Greenwich,LONDON,SE18


In [40]:
df1.shape
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310 entries, 0 to 530
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   borough    310 non-null    object
 1   town       310 non-null    object
 2   post_code  310 non-null    object
dtypes: object(3)
memory usage: 9.7+ KB


In [41]:
# Defining London arcgis geocode function to return latitude and longitude
def get_x_y_uk(address1):
   lat_coords = 0
   lng_coords = 0
   g = geocode(address='{}, London, England, GBR'.format(address1))[0]
   lng_coords = g['location']['x']
   lat_coords = g['location']['y']
   return str(lat_coords) +","+ str(lng_coords)

In [42]:
# Checking sample data
c = get_x_y_uk('SE2')
c

'51.492450000000076,0.12127000000003818'

In [43]:
geo_coordinates_uk = df1['post_code']    
geo_coordinates_uk

0           SE2
1        W3, W4
6           EC3
7           WC2
9          SE20
         ...   
523    IG8, E18
524         IG8
527         N12
528        SE18
530         W12
Name: post_code, Length: 310, dtype: object

In [44]:
# Passing postal codes of london to get the geographical co-ordinates
coordinates_latlng_uk = geo_coordinates_uk.apply(lambda x: get_x_y_uk(x))
coordinates_latlng_uk

0       51.492450000000076,0.12127000000003818
1        51.51324000000005,-0.2674599999999714
6       51.51200000000006,-0.08057999999994081
7       51.51651000000004,-0.11967999999995982
9       51.41009000000008,-0.05682999999993399
                        ...                   
523    51.589770000000044,0.030520000000024083
524      51.50642000000005,-0.1272099999999341
527     51.615920000000074,-0.1767399999999384
528      51.48207000000008,0.07143000000002075
530      51.50645000000003,-0.2369099999999662
Name: post_code, Length: 310, dtype: object

In [45]:
lat_uk = coordinates_latlng_uk.apply(lambda x: x.split(',')[0])
lat_uk

0      51.492450000000076
1       51.51324000000005
6       51.51200000000006
7       51.51651000000004
9       51.41009000000008
              ...        
523    51.589770000000044
524     51.50642000000005
527    51.615920000000074
528     51.48207000000008
530     51.50645000000003
Name: post_code, Length: 310, dtype: object

In [46]:
lng_uk = coordinates_latlng_uk.apply(lambda x: x.split(',')[1])
lng_uk

0       0.12127000000003818
1       -0.2674599999999714
6      -0.08057999999994081
7      -0.11967999999995982
9      -0.05682999999993399
               ...         
523    0.030520000000024083
524     -0.1272099999999341
527     -0.1767399999999384
528     0.07143000000002075
530     -0.2369099999999662
Name: post_code, Length: 310, dtype: object

In [47]:
# Add the geographical co-ordinates of the London Neighbourhoods to the data
london_merged = pd.concat([df1,lat_uk.astype(float), lng_uk.astype(float)], axis=1)
london_merged.columns= ['borough','town','post_code','latitude','longitude']
london_merged

,borough,town,post_code,latitude,longitude
0,"Bexley, Greenwich",LONDON,SE2,51.49245,0.12127
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",51.51324,-0.26746
6,City,LONDON,EC3,51.51200,-0.08058
7,Westminster,LONDON,WC2,51.51651,-0.11968
9,Bromley,LONDON,SE20,51.41009,-0.05683
...,...,...,...,...,...
523,Redbridge,LONDON,"IG8, E18",51.58977,0.03052
524,"Redbridge, Waltham Forest","LONDON, WOODFORD GREEN",IG8,51.50642,-0.12721
527,Barnet,LONDON,N12,51.61592,-0.17674
528,Greenwich,LONDON,SE18,51.48207,0.07143


In [48]:
london_merged.dtypes

borough       object
town          object
post_code     object
latitude     float64
longitude    float64
dtype: object

In [49]:
# Getting the geocode for London to help visualize it on the map
london = geocode(address='London, England, GBR')[0]
london_lng_coords = london['location']['x']
london_lat_coords = london['location']['y']
london_lng_coords
london_lat_coords

51.50642000000005

In [50]:
# neighborhood latitude value
neighborhood_latitude = london_merged['latitude'] 

# neighborhood longitude value
neighborhood_longitude = london_merged['longitude'] 

# neighborhood name
neighborhood_name = london_merged['borough']

In [51]:
# Creating the map of London
map_London = folium.Map(location=[london_lat_coords, london_lng_coords], zoom_start=12)
map_London

# adding markers to map
for latitude, longitude, borough, town in zip(london_merged['latitude'], london_merged['longitude'], london_merged['borough'], london_merged['town']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_London)  
    
map_London

In [52]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [58]:
venues_in_London = getNearbyVenues(names= london_merged['borough'],latitudes= london_merged['latitude'], longitudes =london_merged['longitude'])

Bexley, Greenwich 
Ealing, Hammersmith and Fulham
City
Westminster
Bromley
Islington
Islington
Barnet
Enfield
Wandsworth
Southwark
City
Richmond upon Thames
Barnet
Islington
Wandsworth
Westminster
Bromley
Newham
Ealing
Westminster
Lewisham
Camden
Southwark
Tower Hamlets
Bexley
City
Lewisham
Greenwich
Tower Hamlets
Camden
Haringey
Tower Hamlets
Haringey
Barnet
Brent
Lambeth
Lewisham
Tower Hamlets
Kensington and ChelseaHammersmith and Fulham
Brent
Barnet
Barnet
Southwark
Tower Hamlets
Camden
Tower Hamlets
Waltham Forest
Newham
Islington
Richmond upon Thames
Lewisham
Camden
Westminster
Greenwich
Kensington and Chelsea
Barnet
Westminster
Lewisham
Waltham Forest
Hounslow, Ealing, Hammersmith and Fulham
Brent
Barnet
Lambeth, Wandsworth
Islington
Barnet
Merton
Barnet
Westminster
Barnet, Brent, Camden
Lewisham
Bexley
Haringey
Bromley
Tower Hamlets
Newham
Hackney
Dartford
Islington
Southwark
Lewisham
Brent
Southwark
Ealing
Kensington and Chelsea
Wandsworth
Southwark
Barnet
Newham
Richmond upon 

In [59]:
venues_in_London.shape

(10611, 7)

In [60]:
venues_in_London.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Bexley, Greenwich",51.49245,0.12127,Lesnes Abbey,51.489526,0.125839,Historic Site
1,"Bexley, Greenwich",51.49245,0.12127,Sainsbury's,51.492826,0.120524,Supermarket
2,"Bexley, Greenwich",51.49245,0.12127,Lidl,51.496152,0.118417,Supermarket
3,"Bexley, Greenwich",51.49245,0.12127,Abbey Wood Railway Station (ABW),51.490825,0.123432,Train Station
4,"Bexley, Greenwich",51.49245,0.12127,Bean @ Work,51.491172,0.120649,Coffee Shop


In [61]:
# To see how many Venue Categories are there
venues_in_London.groupby('Venue Category').max()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Adult Boutique,Islington,51.52969,-0.08697,Sh! Women's Erotic Emporium,51.527102,-0.083728
African Restaurant,Westminster,51.52587,-0.08808,Red Sea Restaurant,51.523967,-0.080917
American Restaurant,Waltham Forest,51.63261,0.02795,Spielburger,51.632538,0.024169
Antique Shop,Westminster,51.51651,-0.11968,The London Silver Vaults,51.517145,-0.112608
Arcade,Westminster,51.51651,-0.11968,Novelty Automation,51.519793,-0.117055
...,...,...,...,...,...,...
Wings Joint,Hammersmith and Fulham,51.54187,-0.19795,Wingmans,51.545554,-0.202021
Women's Store,Westminster,51.55457,-0.11478,Vivien of Holloway,51.554454,-0.114362
Xinjiang Restaurant,Southwark,51.47480,-0.09313,Silk Road,51.473820,-0.089284


In [62]:
# One Hot Encoding
London_venue_cat = pd.get_dummies(venues_in_London[['Venue Category']], prefix="", prefix_sep="")
London_venue_cat

,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10607,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
# Add the neighbourhood
London_venue_cat['Neighbourhood'] = venues_in_London['Neighbourhood'] 

# Moving neighborhood column to the first column
fixed_columns = [London_venue_cat.columns[-1]] + list(London_venue_cat.columns[:-1])
London_venue_cat = London_venue_cat[fixed_columns]

London_venue_cat.head()

,Neighbourhood,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
# Group the Neighbourhoods and calculate the mean venue categories value in each Neighbourhood
London_grouped = London_venue_cat.groupby('Neighbourhood').mean().reset_index()
London_grouped.head()

,Neighbourhood,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,Barnet,0.0,0.0,0.007092,0.0,0.0,0.0,0.007092,0.0,0.0,...,0.007092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Barnet, Brent, Camden",0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bexley,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Bexley, Greenwich",0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bexley, Greenwich",0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
# Function to get the top most common venue categories
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
# Take the top 10 to cluster the neighbourhoods
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [67]:
# create a new dataframe for London
neighborhoods_venues_sorted_london = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_london['Neighbourhood'] = London_grouped['Neighbourhood']

for ind in np.arange(London_grouped.shape[0]):
    neighborhoods_venues_sorted_london.iloc[ind, 1:] = return_most_common_venues(London_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_london.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barnet,Coffee Shop,Café,Grocery Store,Italian Restaurant,Pub,Bus Stop,Supermarket,Sushi Restaurant,Pharmacy,Turkish Restaurant
1,"Barnet, Brent, Camden",Bus Station,Gym / Fitness Center,Clothing Store,Supermarket,Zoo Exhibit,Fish & Chips Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant
2,Bexley,Supermarket,Historic Site,Pub,Coffee Shop,Train Station,Convenience Store,Golf Course,Construction & Landscaping,Park,Bus Stop
3,"Bexley, Greenwich",Golf Course,Sports Club,Bus Stop,Convenience Store,Construction & Landscaping,Historic Site,Park,Film Studio,Exhibit,Falafel Restaurant
4,"Bexley, Greenwich",Supermarket,Pub,Train Station,Coffee Shop,Convenience Store,Historic Site,Film Studio,Exhibit,Falafel Restaurant,Farmers Market


In [68]:
# set number of clusters
k_num_clusters = 5

London_grouped_clustering = London_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_london = KMeans(n_clusters=k_num_clusters, random_state=0).fit(London_grouped_clustering)
kmeans_london

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [69]:
# Labelling Clustered Data
kmeans_london.labels_

array([0, 2, 2, 4, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

In [70]:
neighborhoods_venues_sorted_london.insert(0, 'Cluster Labels', kmeans_london.labels_ +1)

In [71]:
london_data = london_merged

london_data = london_data.join(neighborhoods_venues_sorted_london.set_index('Neighbourhood'), on='borough')

london_data.head()

,borough,town,post_code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bexley, Greenwich",LONDON,SE2,51.49245,0.12127,3,Supermarket,Pub,Train Station,Coffee Shop,Convenience Store,Historic Site,Film Studio,Exhibit,Falafel Restaurant,Farmers Market
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",51.51324,-0.26746,4,Grocery Store,Train Station,Breakfast Spot,Park,Indian Restaurant,Dessert Shop,Exhibit,Dance Studio,Food Stand,Food Court
6,City,LONDON,EC3,51.51200,-0.08058,1,Coffee Shop,Hotel,Italian Restaurant,Gym / Fitness Center,Pub,Restaurant,Sandwich Place,French Restaurant,Scenic Lookout,Cocktail Bar
7,Westminster,LONDON,WC2,51.51651,-0.11968,1,Hotel,Coffee Shop,Café,Sandwich Place,Pub,Italian Restaurant,Theater,Restaurant,Burger Joint,French Restaurant
9,Bromley,LONDON,SE20,51.41009,-0.05683,3,Supermarket,Convenience Store,Hotel,Grocery Store,Fast Food Restaurant,Park,Gastropub,Bistro,Sandwich Place,Golf Course


In [72]:
# Drop all the NaN values to prevent data skew
london_data_nonan = london_data.dropna(subset=['Cluster Labels'])

In [92]:
map_clusters_london = folium.Map(location=[london_lat_coords, london_lng_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_data_nonan['latitude'], london_data_nonan['longitude'], london_data_nonan['borough'], london_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters_london)
        
map_clusters_london

<h3 align="left">Cluster 1 </h3>

In [74]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 1, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,LONDON,1,Coffee Shop,Hotel,Italian Restaurant,Gym / Fitness Center,Pub,Restaurant,Sandwich Place,French Restaurant,Scenic Lookout,Cocktail Bar
7,LONDON,1,Hotel,Coffee Shop,Café,Sandwich Place,Pub,Italian Restaurant,Theater,Restaurant,Burger Joint,French Restaurant
10,LONDON,1,Coffee Shop,Pub,Café,Food Truck,Vietnamese Restaurant,Italian Restaurant,Gym / Fitness Center,Park,Cocktail Bar,Breakfast Spot
12,LONDON,1,Coffee Shop,Pub,Café,Food Truck,Vietnamese Restaurant,Italian Restaurant,Gym / Fitness Center,Park,Cocktail Bar,Breakfast Spot
14,"BARNET, LONDON",1,Coffee Shop,Café,Grocery Store,Italian Restaurant,Pub,Bus Stop,Supermarket,Sushi Restaurant,Pharmacy,Turkish Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
523,LONDON,1,Café,Coffee Shop,Grocery Store,Pub,Metro Station,Bar,Bakery,BBQ Joint,Park,Seafood Restaurant
524,"LONDON, WOODFORD GREEN",1,Hotel,Café,Pub,Theater,Garden,Plaza,Monument / Landmark,Restaurant,Art Gallery,Steakhouse
527,LONDON,1,Coffee Shop,Café,Grocery Store,Italian Restaurant,Pub,Bus Stop,Supermarket,Sushi Restaurant,Pharmacy,Turkish Restaurant
528,LONDON,1,Pub,Grocery Store,Bus Stop,Coffee Shop,Indian Restaurant,Historic Site,Fish & Chips Shop,Pier,Turkish Restaurant,Park


<h3 align="left">Cluster 2 </h3>

In [75]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 2, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
379,"HARROW, STANMOREEDGWARE, LONDON",2,Gym,Metro Station,Bakery,Food Stand,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fishing Store,Event Space


<h3 align="left">Cluster 3 </h3>

In [76]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 3, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,LONDON,3,Supermarket,Pub,Train Station,Coffee Shop,Convenience Store,Historic Site,Film Studio,Exhibit,Falafel Restaurant,Farmers Market
9,LONDON,3,Supermarket,Convenience Store,Hotel,Grocery Store,Fast Food Restaurant,Park,Gastropub,Bistro,Sandwich Place,Golf Course
29,"BECKENHAM, LONDON",3,Supermarket,Convenience Store,Hotel,Grocery Store,Fast Food Restaurant,Park,Gastropub,Bistro,Sandwich Place,Golf Course
45,"BEXLEYHEATH, LONDON",3,Supermarket,Historic Site,Pub,Coffee Shop,Train Station,Convenience Store,Golf Course,Construction & Landscaping,Park,Bus Stop
121,LONDON,3,Bus Station,Gym / Fitness Center,Clothing Store,Supermarket,Zoo Exhibit,Fish & Chips Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant
124,LONDON,3,Supermarket,Historic Site,Pub,Coffee Shop,Train Station,Convenience Store,Golf Course,Construction & Landscaping,Park,Bus Stop
127,LONDON,3,Supermarket,Convenience Store,Hotel,Grocery Store,Fast Food Restaurant,Park,Gastropub,Bistro,Sandwich Place,Golf Course
292,"LONDON, SIDCUP",3,Supermarket,Historic Site,Pub,Coffee Shop,Train Station,Convenience Store,Golf Course,Construction & Landscaping,Park,Bus Stop
317,LONDON,3,Supermarket,Convenience Store,Hotel,Grocery Store,Fast Food Restaurant,Park,Gastropub,Bistro,Sandwich Place,Golf Course
361,LONDON,3,Supermarket,Convenience Store,Hotel,Grocery Store,Fast Food Restaurant,Park,Gastropub,Bistro,Sandwich Place,Golf Course


<h3 align="left">Cluster 4 </h3>

In [77]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 4, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,LONDON,4,Grocery Store,Train Station,Breakfast Spot,Park,Indian Restaurant,Dessert Shop,Exhibit,Dance Studio,Food Stand,Food Court


<h3 align="left">Cluster 5 </h3>

In [78]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 5, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
168,"LONDON, WELLING",5,Golf Course,Sports Club,Bus Stop,Convenience Store,Construction & Landscaping,Historic Site,Park,Film Studio,Exhibit,Falafel Restaurant
459,"LONDON, ERITH",5,Golf Course,Sports Club,Bus Stop,Convenience Store,Construction & Landscaping,Historic Site,Park,Film Studio,Exhibit,Falafel Restaurant


<h2 align="left">Results and Discussion </h2>

The neighbourhoods of London are very mulitcultural. There are a lot of different cusines including Indian, Italian, Turkish and Chinese. London seems to take a step further in this direction by having a lot of Restaurants, bars, juice bars, coffee shops, Fish and Chips shop and Breakfast spots. It has a lot of shopping options too with that of the Flea markets, flower shops, fish markets, Fishing stores, clothing stores. The main modes of transport seem to be Buses and trains. For leisure, the neighbourhoods are set up to have lots of parks, golf courses, zoo, gyms and Historic sites.

Overall, the city of London offers a multicultural, diverse and certainly an entertaining experience.

Paris is relatively small in size geographically. It has a wide variety of cusines and eateries including French, Thai, Cambodian, Asian, Chinese etc. There are a lot of hangout spots including many Restaurants and Bars. Paris has a lot of Bistro's. Different means of public transport in Paris which includes buses, bikes, boats or ferries. For leisure and sight seeing, there are a lot of Plazas, Trails, Parks, Historic sites, clothing shops, Art galleries and Museums. Overall, Paris seems like the relaxing vacation spot with a mix of lakes, historic spots and a wide variety of cusines to try out.

<h2 align="left">Conclusion </h2> 

The purpose of this project was to explore the cities of London and Paris and see how attractive it is to potential tourists and migrants. We explored both the cities based on their postal codes and then extrapolated the common venues present in each of the neighbourhoods finally concluding with clustering similar neighbourhoods together.

We could see that each of the neighbourhoods in both the cities have a wide variety of experiences to offer which is unique in it's own way. The cultural diversity is quite evident which also gives the feeling of a sense of inclusion.

Both Paris and London seem to offer a vacation stay or a romantic gateaway with a lot of places to explore, beautiful landscapes and a wide variety of culture.Overall, it's upto the stakeholders to decide which experience they would prefer more and which would more to their liking.